## Import Data

In [1]:
#list the data
!ls Data

averages.csv
fivethirtyeight_partisan_lean_DISTRICTS.csv
fivethirtyeight_partisan_lean_STATES.csv
vote_predictions.csv


In [2]:
!ls

Capstone Project Proposal - 2.pdf  Data Wrangling.ipynb    README.md
Data				   PROPUBLICA_API_KEY.TXT


### Get API working

In [3]:
#Load ProPublica API key without exposing it to github. Request one yourself. They're free.
APIfile = open('./PROPUBLICA_API_KEY.TXT',)
APIkey = APIfile.read().replace('\n', '')
print(type(APIkey))
#print(APIkey)

<class 'str'>


In [4]:
#API request function
import requests

#create API key
headers = {'X-API-Key': APIkey}

#load status codes from API documentation(https://projects.propublica.org/api-docs/congress-api/#errors) into dictionary
status_codes = {
    200:'Successful Connection!',
    400:'Bad Request – Your request is improperly formed',
    403:'Forbidden – Your request did not include an authorization header',
    404:'Not Found – The specified record(s) could not be found',
    406:'Not Acceptable – You requested a format that isn’t json or xml',
    500:'Internal Server Error – We had a problem with our server. Try again later',
    503:'Service Unavailable – The service is currently not working. Please try again later'
    }

#load base url and test endpoint
url = 'https://api.propublica.org/congress/v1'
test_endpoint = '/members/new.json'

def API_Request(end_point, verbose=True):
#send a get request to restful API, print status and return response as a dictionary
    response = requests.get(url+end_point, headers=headers)
    #print the response status
    
    try:
        status = status_codes[response.status_code]
    except: 
        status = ('API Failure, unknown status code: '+str(response.status_code))
        return status
    
    if verbose == True:
        print(status)
        
    return response.json()

#test that the API is working. 
new_members = API_Request(test_endpoint)
print(new_members.keys()) # When the API works this should give the list of attributes per member
print('status:', new_members['status'])
print('no. results:', new_members['results'][0]['num_results'])

Successful Connection!
dict_keys(['status', 'copyright', 'results'])
status: OK
no. results: 20


### The API is working.  Let's import voting data

In [9]:
def API_attempts(endpoint, attempts, verbose=False):
    '''takes and endpoint and requests the API with it for a given number of times'''  
    
    #loop throught the number of attempts
    for attempt in range(attempts):
    
        #if we are already trying again we'll do it loud
        if attempt > 0: 
            verbose = True
            print('Retry:', attempt)
        
        #send endpoint to API function for request
        response = API_Request(endpoint, verbose)
    
        #if we can't extract the status we'll keep trying, and if it's OK we'll stop
        status = 'failed'
        #extract the status of the API request
        try:
            status = response['status']
        except:
            pass
        if status == 'OK':
            break
        else: # if the status is not OK, say so
            print('Status:', status)

    return response, status

#let's create a function that will import vote positions for a given rollcall vote.
def import_vote(congress, chamber, session, roll_call, verbose=False):
    '''imports vote details and member positions of a given roll_call vote
        takes congress number 102-116 for House, 80-116 for Senate
        chamber(house|senate) session 1 for odd 2 for even-numbered years
        returns a dicitonary of vote positions and API status'''
    #construct endpoint for API request
    call_endpoint = '/'+str(congress)+'/'+chamber.lower()+'/sessions/'+str(session)+'/votes/'+str(roll_call)+'.json'
    
    #let's attempt a few times to account for internet burps
    call_response, status = API_attempts(call_endpoint, 3)  
        
    #try to extract and return the vote positions
    try:
        vote = call_response['results']['votes']['vote']
        return vote, status
    except:
        return {}, status 
    

    
#let's create a function that will import a given congressional session
def import_session(congress, chamber, session):
    '''import a congressional session worth of votes using the pro-publica API
        takes the congress number chamber and session as input and returns a data frame'''
    #initialize roll_call and Status
    roll_call = 0 
    status = 'OK'
    
    #announce which congress chamber and session we're importing
    print('Importing ',congress,chamber,session)
    
    #Keep using the function above to import votes as long as the API returns an ok status
    while status == 'OK':
        roll_call += 1 #itererate which roll call we are on
        
        #use the API to import votes
        vote, status = import_vote(congress,chamber,session,roll_call)
                
        #get the positions of a given vote while removing them from the vote dictionary
        try:
            positions = vote.pop('positions')
            votedicts = {}
            for key in list(vote.keys()):
                if type(vote[key]) == dict:
                    votedicts[key] = vote.pop(key)
            #print(votedicts)#debug
            
        except:
            positions = {}
            vote = {}
        #Make a series out of the votes
        vote_df = pd.Series(vote)
        #print(vote_df) #debug
        
        #Make a dataframe out of the positions
        call_df = pd.DataFrame(positions)
        
        #replace the column called 'vote_position' with the roll_call number we are on.
        call_df = call_df.rename(columns={'vote_position':roll_call})
        
        #find the length of the positions dataframe
        dflng = len(call_df)
        
        #make sure the lenth of the data frame fits the critera for a congressional house
        if dflng > 0 and dflng < 440:
            try: #merge this dataframe with the previous so the votes stack up 
                dfp = pd.merge(dfp, call_df, on=mergelst, how='outer')
                print('.', end='') #print a little dot to show we are making progress
                
                #stack the vote dataframes
                dfv = pd.concat([dfv, vote_df], axis=1)
            except:
                #if the above fails, it's probably because there wasn't a data set to merge into.
                #we'll initialize it here
                print('Initalizing df', roll_call)
                print('API status:', status)
                dfp = call_df
                dfv = vote_df
                #these are the columns we'll merge on (everthing except the vote position)
                mergelst = dfp.columns.tolist()[:-1]
                print('merging on:',mergelst)
        else: #if the vote postions don't fit the size critera we'll skip them to avoid screwing up the df
            print('(Size) skip vote', roll_call)
   
    dfv = dfv.transpose()
    dfp = dfp.set_index(['party','state','member_id','name','dw_nominate']).transpose()
    
    
    return dfv.join(dfp, on='roll_call').set_index(['congress','chamber','session','roll_call'])

def positions_df(metalist):
    '''return a dataframe of postions for a given list of vote metadata'''
    
    #initialize dataframe
    dfp = pd.DataFrame()
    
    #for every position in the list
    for congress, chamber, session, roll_call in metalist:
        
        #request the roll_call vote from the API
        vote, status = import_vote(congress, chamber, session, roll_call)
        
        #extract the positions from the rollcall vote data
        positions = vote.pop('positions')
        
        #turn the postions into a dataframe
        call_positions = pd.DataFrame(positions).drop_duplicates()
        
        #if the output dataframe isn't empty and the size is right,
        if not dfp.empty:
            
            #merge the positions for that call into it
            dfp = pd.merge(dfp, call_positions, on=mergelst, how='outer')
            
            #let us know it's working
            print('.', end='')
            
        #if it is empty
        elif dfp.empty: 
            #make it equal to call_positions
            dfp = call_positions
            
            #make a list of the columns that will be used to merge the call positions
            mergelst = dfp.columns.tolist()[:-1]
            
            #move party and state to front of merglist
            for x in ['state','party']:
                mergelst.insert(0, mergelst.pop(mergelst.index(x)))
            
            #alert that we've started
            print('df created', end=' ')
     
    
    #create a multindex of partymembers
    dfp = dfp.set_index(mergelst).transpose()
    
    #after transposing use the metalist as the index
    dfp.index = metalist
    
    return dfp 

def import_month(year, chamber, month):
    '''import vote metadata for a given year or month and return a dataframe '''
    
    #construct endpoint
    call_endpoint = '/'+chamber.lower()+'/votes/'+str(year)+'/'+str(month)+'.json'
    
    #make 3 attempts at calling the API with the endpoint
    response, status = API_attempts(call_endpoint, 3)
    
    #Extract the votes from the response
    votes = response['results']['votes']

    #create a dataframe out of the votes
    votesdf = pd.DataFrame(votes)
    
    if not votesdf.empty:
        votesdf = votesdf.set_index(['congress','chamber','session','roll_call']).sort_index()
    
    #return the months dataframe
    return votesdf

def import_year(year, chamber):
    '''import votes based on dates, return a dataframe of appended votes'''
    
    
    #So we know we started:
    print('importing', chamber, year, end=' ')
    #loop through the months 
    for month in range(1,13):
        
        #import the first month
        if month == 1:
            yeardf = import_month(year, chamber, month)
       
        #import the month of votes as a df and append it to the list 
        else:
            df = import_month(year, chamber, month)
            if not df.empty:
                yeardf = yeardf.append(df)
            else:
                print('skip', end=' ')
        #print something so we know it's working
        print(month, end=' ')
        
    #return the dataframe
    return yeardf
    

#test (remove when working)
print('testing import with latest session:')
#latest_session_test_1_df = import_session(116, 'House', 1)
#latest_session_test_1_df
senate_2019_test = import_year(2019, 'senate')
positions = positions_df(senate_2019_test.index)
positions

testing import with latest session:
importing senate 2019 1 2 3 4 5 skip 6 skip 7 skip 8 skip 9 skip 10 skip 11 skip 12 df created ...................................................................................................................

party                                            R             D  \
state                                           TN            WI   
dw_nominate                                  0.323        -0.521   
member_id                                  A000360       B001230   
name                               Lamar Alexander Tammy Baldwin   
congress chamber session roll_call                                 
116      Senate  1       1                     Yes            No   
                         2                     Yes            No   
                         3                     Yes            No   
                         4                     Yes            No   
                         5                      No           Yes   
                         6                      No           Yes   
                         7              Not Voting            No   
                         8                      No           Yes   
                         9                     Yes            No   
                         10                    Yes           Yes   
                         11                    Yes            No   
                         12                    Yes            No   
                         13             Not Voting            No   
                         14                    Yes            No   
                         15                    Yes            No   
                         16                    Yes            No   
                         17                    Yes           Yes   
                         18                    Yes           Yes   
                         19                    Yes           Yes   
                         20                    Yes           Yes   
                         21                    Yes           Yes   
                         22                    Yes           Yes   
                         23                    Yes            No   
                         24                    Yes            No   
                         25                    Yes           Yes   
                         26                    Yes           Yes   
                         27                    Yes            No   
                         28                    Yes            No   
                         29                    Yes            No   
                         30                    Yes           Yes   
...                                            ...           ...   
                         87                    Yes            No   
                         88                    Yes           Yes   
                         89                    Yes           Yes   
                         90                    Yes            No   
                         91                    Yes           Yes   
                         92                    Yes           Yes   
                         93                    Yes            No   
                         94                     No           Yes   
                         95                    Yes            No   
                         96                    Yes           Yes   
                         97                    Yes           Yes   
                         98                    Yes           Yes   
                         99                    Yes            No   
                         100                   Yes           Yes   
                         101                   Yes           Yes   
                         102                   Yes           Yes   
                         103                   Yes            No   
                         104                   Yes            No   
                         105                   Yes            No   
                         106                   Yes            No   
                         107                   Yes            No   
                         108                   Yes      

In [10]:
house_2019_test = import_year(2019, 'house')
house_2019_ptest = positions_df(house_2019_test.index)
house_2019_ptest

importing house 2019 1 2 3 4 5 skip 6 skip 7 skip 8 skip 9 skip 10 skip 11 skip 12 df created ........................................................................................................................................................................................................................

party                                          R          D  \
state                                         LA         NC   
district                                       5         12   
dw_nominate                                0.514     -0.469   
member_id                                A000374    A000370   
name                               Ralph Abraham Alma Adams   
congress chamber session roll_call                            
116      House   1       1               Present    Present   
                         2              McCarthy     Pelosi   
                         3                    No        Yes   
                         4                    No        Yes   
                         5                   Yes         No   
                         6                    No        Yes   
                         7                    No        Yes   
                         8                   Yes         No   
                         9                    No        Yes   
                         10                  Yes         No   
                         11                   No        Yes   
                         12                  Yes        Yes   
                         13                  Yes        Yes   
                         14                  Yes        Yes   
                         15                   No        Yes   
                         16                   No        Yes   
                         17                   No        Yes   
                         18                  Yes        Yes   
                         19                   No        Yes   
                         20                  Yes         No   
                         21                   No        Yes   
                         22                  Yes         No   
                         23                   No        Yes   
                         24                  Yes         No   
                         25                   No        Yes   
                         26           Not Voting         No   
                         27           Not Voting        Yes   
                         28           Not Voting        Yes   
                         29           Not Voting        Yes   
                         30                  Yes        Yes   
...                                          ...        ...   
                         188                 Yes        Yes   
                         189                  No        Yes   
                         190                  No        Yes   
                         191          Not Voting        Yes   
                         192          Not Voting         No   
                         193          Not Voting        Yes   
                         194          Not Voting        Yes   
                         195          Not Voting         No   
                         196          Not Voting        Yes   
                         197          Not Voting        Yes   
                         198          Not Voting        Yes   
                         199          Not Voting        Yes   
                         200          Not Voting        Yes   
                         201          Not Voting         No   
                         202          Not Voting        Yes   
                         203          Not Voting        Yes   
                         204          Not Voting        Yes   
                         205          Not Voting        Yes   
                         206          Not Voting        Yes   
                         207          Not Voting        Yes   
                         208          Not Voting        Yes   
                         209          Not Voting        Yes   
                         210          Not Voting         No   
                         211          Not Voting        Yes   
                         212          Not Voting        Yes   
                         213   

In [11]:
house_2019_ptest.sort_index(axis='columns', level = 'dw_nominate')

party                                         D                       \
state                                        AZ          CA            
district                                      9          10       21   
dw_nominate                                 NaN         NaN      NaN   
member_id                               S001211     H001090  C001124   
name                               Greg Stanton Josh Harder   TJ Cox   
congress chamber session roll_call                                     
116      House   1       1              Present     Present  Present   
                         2               Pelosi      Pelosi   Pelosi   
                         3                  Yes         Yes      Yes   
                         4                  Yes         Yes      Yes   
                         5                   No          No       No   
                         6                  Yes         Yes      Yes   
                         7                  Yes         Yes      Yes   
                         8                   No          No       No   
                         9                  Yes         Yes      Yes   
                         10                  No          No       No   
                         11                 Yes         Yes      Yes   
                         12                 Yes         Yes      Yes   
                         13                 Yes         Yes      Yes   
                         14                 Yes         Yes      Yes   
                         15                 Yes         Yes      Yes   
                         16                 Yes         Yes      Yes   
                         17                 Yes         Yes      Yes   
                         18                 Yes         Yes      Yes   
                         19                 Yes         Yes      Yes   
                         20                  No          No       No   
                         21                 Yes         Yes      Yes   
                         22                  No          No       No   
                         23                 Yes         Yes      Yes   
                         24                  No          No       No   
                         25                 Yes         Yes      Yes   
                         26                  No          No       No   
                         27                 Yes         Yes      Yes   
                         28                 Yes         Yes      Yes   
                         29                 Yes         Yes      Yes   
                         30                 Yes         Yes      Yes   
...                                         ...         ...      ...   
                         188                Yes         Yes      Yes   
                         189                Yes         Yes      Yes   
                         190                Yes         Yes      Yes   
                         191                Yes         Yes      Yes   
                         192                 No          No       No   
                         193                Yes         Yes      Yes   
                         194                Yes         Yes      Yes   
                         195                 No          No       No   
                         196                Yes         Yes      Yes   
                         197                Yes         Yes      Yes   
                         198                Yes         Yes      Yes   
                         199                Yes         Yes      Yes   
                         200                Yes         Yes      Yes   
                         201                 No          No       No   
                         202                Yes         Yes      Yes   
                         203                Yes         Yes      Yes   
                         204                Yes         Yes      Yes   
                         205            

In [ ]:
#import vote postitions for 115th senate, session 1
senate_115_1_df = import_session(115, 'senate', 1)
senate_115_1_df

In [21]:
_

,amendment,bill,chamber,congress,date,democratic,description,document_number,document_title,independent,...,result,roll_call,session,source,tie_breaker,tie_breaker_vote,time,total,url,vote_type
api_uri,NaN,https://api.propublica.org/congress/v1/116/bil...,Senate,116,2019-01-08,NaN,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
bill_id,NaN,s1-116,Senate,116,2019-01-08,NaN,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
latest_action,NaN,Held at the desk.,Senate,116,2019-01-08,NaN,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
majority_position,NaN,NaN,Senate,116,2019-01-08,No,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
no,NaN,NaN,Senate,116,2019-01-08,41,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,2.0,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,44.0,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
not_voting,NaN,NaN,Senate,116,2019-01-08,0,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,0.0,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,0.0,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
number,NaN,S.1,Senate,116,2019-01-08,NaN,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
present,NaN,NaN,Senate,116,2019-01-08,0,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,0.0,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,0.0,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
short_title,NaN,Strengthening America's Security in the Middle...,Senate,116,2019-01-08,NaN,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5
title,NaN,A bill to make improvements to certain defense...,Senate,116,2019-01-08,NaN,A bill to make improvements to certain defense...,1,A bill to make improvements to certain defense...,NaN,...,Cloture on the Motion to Proceed Rejected,1,1,https://www.senate.gov/legislative/LIS/roll_ca...,,,17:39:00,NaN,https://www.senate.gov/legislative/LIS/roll_ca...,3/5


In [23]:
#import vote poisiotns for 115th house, session 1
house_115_1_df = import_session(115, 'house', 1)
house_115_1_df

Importing  115 house 1
(Size) skip vote 1
Initalizing df 2
API status: OK
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................(Size) skip vote 711


,member_id,name,party,state,vote_position,vote_position_3,vote_position_4,vote_position_5,vote_position_6,vote_position_7,...,vote_position_701,vote_position_702,vote_position_703,vote_position_704,vote_position_705,vote_position_706,vote_position_707,vote_position_708,vote_position_709,vote_position_710
0,A000374,Ralph Abraham,R,LA,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
1,A000370,Alma Adams,D,NC,Pelosi,Not Voting,No,Yes,No,Yes,...,Yes,No,No,No,No,Yes,No,No,Yes,Yes
2,A000055,Robert B. Aderholt,R,AL,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
3,A000371,Pete Aguilar,D,CA,Pelosi,No,No,Yes,No,Yes,...,Yes,No,No,No,No,Yes,No,No,Yes,Yes
4,A000372,Rick Allen,R,GA,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
5,A000367,Justin Amash,R,MI,Ryan (WI),Yes,Yes,No,No,No,...,No,No,No,Yes,No,Yes,No,No,No,Yes
6,A000369,Mark Amodei,R,NV,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
7,A000375,Jodey Arrington,R,TX,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Not Voting
8,B001291,Brian Babin,R,TX,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
9,B001298,Don Bacon,R,NE,Ryan (WI),Yes,Yes,No,Yes,No,...,No,Yes,Yes,Yes,Yes,Yes,Not Voting,Yes,Yes,Yes


In [41]:
positions, status = import_positions(115,'house',1,3)
pd.DataFrame(positions)

,district,dw_nominate,member_id,name,party,state,vote_position
0,5,0.508,A000374,Ralph Abraham,R,LA,Yes
1,12,-0.469,A000370,Alma Adams,D,NC,Not Voting
2,4,0.361,A000055,Robert B. Aderholt,R,AL,Yes
3,31,-0.285,A000371,Pete Aguilar,D,CA,No
4,12,0.611,A000372,Rick Allen,R,GA,Yes
5,3,0.648,A000367,Justin Amash,R,MI,Yes
6,2,0.375,A000369,Mark Amodei,R,NV,Yes
7,19,0.549,A000375,Jodey Arrington,R,TX,Yes
8,36,0.693,B001291,Brian Babin,R,TX,Yes
9,2,0.432,B001298,Don Bacon,R,NE,Yes


In [36]:
test = pd.DataFrame(positions)
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 7 columns):
district         1292 non-null object
dw_nominate      1292 non-null float64
member_id        1292 non-null object
name             1292 non-null object
party            1292 non-null object
state            1292 non-null object
vote_position    1292 non-null object
dtypes: float64(1), object(6)
memory usage: 70.7+ KB


In [37]:
names = test['name'].tolist()
nameset = set(names)
len(nameset)

436

In [39]:
ids = test['member_id'].tolist()
idset = set(ids)
len(idset)

436